In [436]:
import pandas as pd
import numpy as np 
import os, sys
sys.path.append(os.getcwd())
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
import seaborn as sns
from statsmodels.tsa.holtwinters import ExponentialSmoothing




In [437]:
cierre_agricola =pd.read_csv('/home/sayi/NC.c20-64-m-data-bi/DataSets/cierre_agricola.csv', encoding='latin1')
dfPP = pd.read_csv('/home/sayi/NC.c20-64-m-data-bi/DataSets/PP_Chiapas.csv', encoding='latin1')
dfPT = pd.read_csv('/home/sayi/NC.c20-64-m-data-bi/DataSets/dfPT.csv', encoding='latin1')
dfS = pd.read_csv('/home/sayi/NC.c20-64-m-data-bi/DataSets/Suelos_Chiapas.csv', encoding='latin1')

In [438]:
df = cierre_agricola
# Filtrar el DataFrame para excluir las filas donde 'Nomcultivo' es 'Pastos y praderas'
df = df[df['Nomcultivo'] != 'Pastos y praderas']

df.rename(columns= {'Anio': 'Año' ,'Nomestado': 'Estado', 'Nommunicipio': 'Municipio' }, inplace=True)

# Verifica que se haya eliminado correctamente
print(df['Nomcultivo'].unique())

['MaÃ\xadz grano' 'SandÃ\xada' 'Sorgo grano' 'Cacahuate' 'Aguacate'
 'Ciruela' 'LimÃ³n' 'Mango' 'Naranja' 'Papaya' 'Frijol' 'CafÃ© cereza'
 'Mandarina' 'PlÃ¡tano' 'Cebolla' 'Chile verde' 'MelÃ³n' 'Guayaba' 'Mamey'
 'Zapote' 'Pimienta' 'Cacao' 'Nanche' 'Tomate rojo (jitomate)'
 'Tomate verde' 'MaraÃ±Ã³n' 'Tamarindo' 'CaÃ±a de azÃºcar' 'Durazno'
 'Manzana' 'Col (repollo)' 'Papa' 'PerÃ³n' 'Tejocote' 'Trigo grano' 'Lima'
 'Toronja (pomelo)' 'Zarzamora' 'Garbanzo grano' 'Pepino' 'Arroz palay'
 'Chile seco' 'Girasol' 'Hule hevea' 'Palma africana o de aceite'
 'Macadamia' 'Tabaco' 'AjonjolÃ\xad' 'Soya' 'Copra' 'PiÃ±a'
 'Calabaza semilla o chihua' 'Vainilla' 'Camote'
 'Sorgo forrajero en verde' 'Hortalizas' 'RambutÃ¡n'
 'CaÃ±a de azÃºcar piloncillo' 'Palma camedor' 'Pera' 'PiÃ±Ã³n'
 'Semilla de caÃ±a de azÃºcar' 'Litchi' 'Semilla de soya' 'Stevia'
 'Semilla de maÃ\xadz grano' 'MangostÃ¡n' 'Elote' 'Chayote' 'Granada'
 'Crisantemo (gruesa)' 'Lilium (gruesa)' 'Palma camedor (gruesa)']


/tmp/ipykernel_901/1176532396.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns= {'Anio': 'Año' ,'Nomestado': 'Estado', 'Nommunicipio': 'Municipio' }, inplace=True)


In [439]:
sin_pasto = df.groupby('Año').agg({
    'Volumenproduccion': 'sum',
    'Valorproduccion': 'sum', 
}).reset_index()

In [440]:
dfPT.rename(columns={'AÃ±o': 'Año', 'Anual': 'PromTemp_anual'}, inplace=True)
dfPP.rename(columns={'AÃ±o': 'Año', 'Anual': 'Prec_anual'}, inplace=True)

In [441]:
dfPP = dfPP[['Año','Prec_anual']]
dfPT = dfPT[['Año','PromTemp_anual']]

In [442]:
df_pred = pd.merge(sin_pasto, dfPP, on=['Año'], how='left')


In [443]:
df_pred = pd.merge(df_pred, dfPT, on=['Año'], how='left')

df_pred.head()

,Año,Volumenproduccion,Valorproduccion,Prec_anual,PromTemp_anual
0,2003,6417346.15,8.636436e+09,1935.6,24.0
1,2004,6195827.08,8.741246e+09,1713.8,24.0
2,2005,6210077.83,8.660470e+09,2190.7,24.7
3,2006,5971190.30,1.021940e+10,2354.2,25.4
4,2007,5963631.89,1.125255e+10,2302.6,25.0


In [444]:
column_names = df_pred.columns
print(column_names)

Index(['Año', 'Volumenproduccion', 'Valorproduccion', 'Prec_anual',
       'PromTemp_anual'],
      dtype='object')


In [445]:
df_pred.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Año                21 non-null     int64  
 1   Volumenproduccion  21 non-null     float64
 2   Valorproduccion    21 non-null     float64
 3   Prec_anual         21 non-null     float64
 4   PromTemp_anual     21 non-null     float64
dtypes: float64(4), int64(1)
memory usage: 972.0 bytes


In [446]:
df_pred.describe()

,Año,Volumenproduccion,Valorproduccion,Prec_anual,PromTemp_anual
count,21.000000,2.100000e+01,2.100000e+01,21.000000,21.000000
mean,2013.000000,6.488802e+06,1.484989e+10,2032.838095,24.909524
std,6.204837,3.744217e+05,3.850798e+09,294.304284,0.590682
min,2003.000000,5.936284e+06,8.636436e+09,1614.200000,24.000000
25%,2008.000000,6.210078e+06,1.236714e+10,1767.400000,24.500000
50%,2013.000000,6.416112e+06,1.529533e+10,1994.000000,24.900000
75%,2018.000000,6.721255e+06,1.772723e+10,2278.900000,25.400000
max,2023.000000,7.264487e+06,2.181591e+10,2730.300000,26.100000


In [447]:
# Seleccionar las columnas de tipo object
object_cols = df_pred.select_dtypes(include='object').columns

# Eliminar las columnas de tipo object
df_pred = df_pred.drop(columns=object_cols)

# Verificar el resultado
print(df_pred.head())

    Año  Volumenproduccion  Valorproduccion  Prec_anual  PromTemp_anual
0  2003         6417346.15     8.636436e+09      1935.6            24.0
1  2004         6195827.08     8.741246e+09      1713.8            24.0
2  2005         6210077.83     8.660470e+09      2190.7            24.7
3  2006         5971190.30     1.021940e+10      2354.2            25.4
4  2007         5963631.89     1.125255e+10      2302.6            25.0


In [448]:
column_names = df_pred.columns
print(column_names)

Index(['Año', 'Volumenproduccion', 'Valorproduccion', 'Prec_anual',
       'PromTemp_anual'],
      dtype='object')


In [449]:
# Identificar columnas que comienzan con 'Id'
id_cols = [col for col in df_pred.columns if col.startswith(('Id', 'ID'))]
# Eliminar las columnas que comienzan con 'Id'
df_pred = df_pred.drop(columns=id_cols)

# Verificar el resultado
print(df_pred.head())

    Año  Volumenproduccion  Valorproduccion  Prec_anual  PromTemp_anual
0  2003         6417346.15     8.636436e+09      1935.6            24.0
1  2004         6195827.08     8.741246e+09      1713.8            24.0
2  2005         6210077.83     8.660470e+09      2190.7            24.7
3  2006         5971190.30     1.021940e+10      2354.2            25.4
4  2007         5963631.89     1.125255e+10      2302.6            25.0


In [450]:
column_names = df_pred.columns
print(column_names)

Index(['Año', 'Volumenproduccion', 'Valorproduccion', 'Prec_anual',
       'PromTemp_anual'],
      dtype='object')


In [451]:
df_pred.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Año                21 non-null     int64  
 1   Volumenproduccion  21 non-null     float64
 2   Valorproduccion    21 non-null     float64
 3   Prec_anual         21 non-null     float64
 4   PromTemp_anual     21 non-null     float64
dtypes: float64(4), int64(1)
memory usage: 972.0 bytes


In [452]:

X = df_pred.drop(columns=['Volumenproduccion'])
y = df_pred['Volumenproduccion']

# Ajustar el modelo Random Forest
rf = RandomForestRegressor()
rf.fit(X, y)

# Importancia de las variables
importancias = rf.feature_importances_
variables = X.columns
for var, imp in zip(variables, importancias):
    print(f'{var}: {imp:.4f}')

Año: 0.2397
Valorproduccion: 0.5826
Prec_anual: 0.0801
PromTemp_anual: 0.0975


In [453]:
# Calcular la matriz de correlación
corr_matrix = df_pred.corr()

corr_with_volprod = corr_matrix['Volumenproduccion']

sorted_correlation = corr_with_volprod.sort_values(ascending = False)

print(sorted_correlation)

Volumenproduccion    1.000000
Valorproduccion      0.755757
Año                  0.639268
PromTemp_anual       0.220343
Prec_anual          -0.253352
Name: Volumenproduccion, dtype: float64


In [454]:

df_pred['Año'] = pd.to_datetime(df_pred['Año'], format='%Y')
df_pred.set_index('Año', inplace=True)

# Definir la serie temporal
serie = df_pred['Volumenproduccion']

# División en entrenamiento y prueba (últimos 5 años para prueba)
train = serie.iloc[:-5]
test = serie.iloc[-5:]

# Ajustar el modelo SARIMA
modelo = SARIMAX(train, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
modelo_ajustado = modelo.fit(disp=False)

# Predicción en el conjunto de prueba
predicciones = modelo_ajustado.forecast(steps=len(test))

# Mostrar las predicciones y los valores reales
print(test)
print(predicciones)

#Evaluar el modelo
mae = mean_absolute_error(test, predicciones)
rmse = np.sqrt(mean_squared_error(test, predicciones))

print(f'MAE: {mae:.2f}')
print(f'RMSE: {rmse:.2f}')


# Mean Absolute Percentage Error (MAPE)
mape = np.mean(np.abs((test - predicciones) / test)) * 100
print(f'MAPE: {mape:.2f}%')

Año
2019-01-01    7081752.81
2020-01-01    6414344.63
2021-01-01    6180038.74
2022-01-01    7264486.56
2023-01-01    6991472.40
Name: Volumenproduccion, dtype: float64
2019-01-01    6.695235e+06
2020-01-01    6.735518e+06
2021-01-01    6.690946e+06
2022-01-01    6.737121e+06
2023-01-01    6.796898e+06
Freq: YS-JAN, Name: predicted_mean, dtype: float64
MAE: 388107.64
RMSE: 407320.01
MAPE: 5.75%


/home/sayi/NC.c20-64-m-data-bi/venv/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency YS-JAN will be used.
  self._init_dates(dates, freq)
/home/sayi/NC.c20-64-m-data-bi/venv/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency YS-JAN will be used.
  self._init_dates(dates, freq)
/home/sayi/NC.c20-64-m-data-bi/venv/lib/python3.11/site-packages/statsmodels/tsa/statespace/sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for ARMA and trend. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
/home/sayi/NC.c20-64-m-data-bi/venv/lib/python3.11/site-packages/statsmodels/tsa/statespace/sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for 